In [2]:
import pandas as pd
from pprint import pprint
import sys
import os
import time

REPO_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(REPO_PATH)

from github_api import get_commit_message, get_commit_diff

Get code diff and commit messages for each row in ApacheJit

In [3]:
apachejit_df = pd.read_csv("../datasets/jit_dp/apachejit_train.csv")
apachejit_list = apachejit_df.to_dict(orient='records')

jit_with_diff_list = []

OWNER = "apache"
MAX_RETRIES = 3
RETRY_DELAY = 2  # seconds
REQUEST_DELAY = 0.5  # 0.5s = 2 req/sec

for commit in apachejit_list:
    project = commit.get('project')
    project_parts = project.split("/")
    repo = project_parts[1]
    commit_hash = commit.get('commit_id')

    retries = 0
    success = False

    while retries < MAX_RETRIES and not success:
        try:
            message = get_commit_message(OWNER, repo, commit_hash)
            diff = get_commit_diff(OWNER, repo, commit_hash)
            success = True
            time.sleep(REQUEST_DELAY) 
        except Exception as e:
            print(f"Error on attempt {retries + 1} for commit {commit_hash}: {e}")
            retries += 1
            if retries < MAX_RETRIES:
                time.sleep(RETRY_DELAY)
            else:
                print(f"❌ Skipping commit {commit_hash} after {MAX_RETRIES} failed attempts.")
                break

    if not success:
        continue

    commit_with_diff = commit
    commit_with_diff['commit_message'] = message
    commit_with_diff['diff'] = diff
    jit_with_diff_list.append(commit_with_diff)

# Save to JSONL
jit_with_diff_df = pd.DataFrame(jit_with_diff_list)
jit_with_diff_df.to_json("../datasets/jit_dp/apachejit_with_diff.jsonl", orient="records", lines=True)

Skipping commit due {'commit_id': '9164c6319ec64adaba540760606d13a35495bacf', 'project': 'apache/hadoop-hdfs', 'buggy': True, 'fix': False, 'year': 2009, 'author_date': 1253053101, 'la': 238, 'ld': 0, 'nf': 3, 'nd': 2, 'ns': 2, 'ent': 0.244283142745731, 'ndev': 5.66666666666667, 'age': 39.6666666666667, 'nuc': 27.6666666666667, 'aexp': 17, 'arexp': 17.0, 'asexp': 1.9969711303710904} to error: Exception: ❌ Failed to get commit message: 422 {"message":"No commit found for SHA: 9164c6319ec64adaba540760606d13a35495bacf","documentation_url":"https://docs.github.com/rest/commits/commits#get-a-commit","status":"422"}
Skipping commit due {'commit_id': 'e8d9bf47ab0c461de204cd8e8facec676f4bbdd2', 'project': 'apache/hadoop-hdfs', 'buggy': True, 'fix': False, 'year': 2009, 'author_date': 1260907160, 'la': 567, 'ld': 415, 'nf': 7, 'nd': 3, 'ns': 2, 'ent': 1.5551537733506495, 'ndev': 2.71428571428571, 'age': 75.1428571428571, 'nuc': 23.285714285714302, 'aexp': 11, 'arexp': 11.0, 'asexp': 1.229492187